In [16]:
import math
import numpy as np
import matplotlib.pyplot as plt


# Metabook pg.76 for xcg estimations 

L_f = 40 # ft - fuselage length
W_f = 5 # 6.2 # ft - fuselage width
C_VT = 0.04 # vertical tail volume coefficient - fixed
#L_VT = 15 # L_f - 11.7 # ft 31.65 # ft vertical tail moment arm
#S_VT = 100 # vertical tail planform area
b_W = 60 # 60 #56 # ft # 60.90 # wing span (tip-to-tip) - fixed
C_HT = 0.5 # horizontal tail volume coefficient - fixed
#L_HT = 15 # L_f - 11.7  # horizontal tail at the same location as vertical tail
# ft 32.95 # ft horizontal tail moment arm
#S_HT = 50 # horizontal tail planform area
c_W = 7 # ft 8.75 # wing mean chord - fixed
S_W = b_W*c_W # ft^2 # 427.8 # wing area - fixed


# Empennage
L_vt = 1 + 17.651 # ft

# Wing
b_w = b_W # ft
c_w = c_W # ft
S_w = S_W # ft^2

# Tail
L_ht = 1 + 17.375 # Calcuated later in the code

# Estimated Vertical Tail Area (ft^2)
S_vt = (C_VT * b_w * S_w) / L_vt 

# Estimated Horizontal Tail Area (ft^2)
S_ht = (C_HT * c_w * S_w) / L_ht

# Actual vertical and horizontal tail areas (ft^2)
S_vt_actual = 42 # ft^2 - actual vertical tail area from design or from AT602 data (?)
S_ht_actual = 85.5 # ft^2 - actual horizontal tail area from design or from AT602 data (?)
S_vt_actual = S_vt_actual
S_ht_actual = S_ht_actual

print("Estimated Vertical Tail Area (ft^2): " + str(np.round(S_vt, 2)))
print("Estimated Horizontal Tail Area (ft^2): " + str(np.round(S_ht, 2)))
print("Actual Vertical Tail Area (ft^2): " + str(np.round(S_vt_actual, 2)))
print("Actual Horizontal Tail Area (ft^2): " + str(np.round(S_ht_actual, 2)))

Estimated Vertical Tail Area (ft^2): 54.05
Estimated Horizontal Tail Area (ft^2): 80.0
Actual Vertical Tail Area (ft^2): 42
Actual Horizontal Tail Area (ft^2): 85.5


In [17]:
# Longitudinal Stability (SM Estimations)

# MAC of a rectangular wing
#c_tip = 6
#c_root = 6

#taper_ratio = c_tip / c_root # pg. 61 of Metabook

MAC_w = S_w / b_w # mean aerodynamic chord

c_root = 7.5

taper_ratio = 0.4133 # 3.1 / 7.5
# MAC_t = (2/3) * (c_root * (1 + taper_ratio + taper_ratio**2) / (1 + taper_ratio)) # 4.5 # mean aerodynamic chord of vertical tail
MAC_t = 4.5 # horizontal tail 
# print(str(c_w / b_w)) --> 0.1 hence Kf = 0.115 - pg. 89 of Metabook

# the quarter point of the MAC is the approximate aerodynamic center of the wing - pg. 86 Metabook

AC_w = 0.25 * MAC_w  # aerodynamic center of wing
AC_t = 0.25 * MAC_t # aerodynamic center of tail

# Horizontal Tail (ft)
#L_ht = L_HT
L_ht = 18 - AC_w + AC_t

print(L_ht)

# dClw_da = 0.16 # wing lift curve slope
# dClt_da = 0.1 # tail lift curve slope
AR_w = b_w / c_w # 8.67 # aspect ratio of the wing - fixed
AR_t = 19 / 4.5 # 4.5 # aspect ratio of the tail - fixed

# Constants to account for the difference between the 
# theoretical section lift curve slope of 2pi and the actual value

eta_w = 0.97 # wing section lift curve difference 
eta_t = 0.97 # tail section lift curve difference 

del_w = 2 # 31.6 # sweep angle (deg)
lambda_w = np.deg2rad(del_w)

del_t = 0 # 35 # sweep angle (deg)
lambda_t = np.deg2rad(del_t)

Ma = 0.2772 # 0.87 # mach number for cruise speed

# Lift curve slope of the wing (1/rad)
CL_a_w = (2 * np.pi * AR_w) / (((2) + (np.sqrt(((( AR_w / eta_w )**2) * (1 + (np.tan(lambda_w))**2 - Ma**2)) + (4)))))

# Lift curve slope of the tail (1/rad)
CL_a_t0 = (2 * np.pi * AR_t) / (((2) + (np.sqrt(((( AR_t / eta_t )**2) * (1 + (np.tan(lambda_t))**2 - Ma**2)) + (4)))))

print("Lift curve slope of the wing (1/radians): " + str(round(CL_a_w, 4)))
print("Lift curve slope of the tail (1/radians): " + str(round(CL_a_t0, 4)))

17.375
Lift curve slope of the wing (1/radians): 5.0201
Lift curve slope of the tail (1/radians): 3.9978


In [18]:
# Downwash effects

de_da = 2 * CL_a_w / (np.pi * AR_w)
print("Downwash (1/radians): " + str(round(de_da, 4)))

Downwash (1/radians): 0.3729


In [19]:
# Correcting for downwash

Cl_a_t = CL_a_t0 * (1 - de_da) * eta_t
print("Lift curve slope of the horizontal tail corrected for downwash (1/radians): " + str(round(Cl_a_t, 4)))

Lift curve slope of the horizontal tail corrected for downwash (1/radians): 2.432


In [20]:

# Fuselage

K_f = 0.688 # emperical factor (estimated)

Lf = L_f 
Wf = W_f 

dCmf_dCL = (K_f * (Wf ** 2) * Lf) / (S_w * MAC_w * CL_a_w)
# print(dCmf_dCL)

x_cg = 13.5 # 35 * 0.33 (approximately) # ft - Aircraft center of gravity assumed
x_25MAC = 0.25 * MAC_w + 9.5 # ft - Distance from nose to 25% MAC assumed

# Calcuating x_np - pg. 87 of metabook OR slides lec 11 slide 31
# From the nose, the wing starts 9.5 feet to the right of the nose
x_np = 13.5 + MAC_w * (((L_vt * S_ht) / (MAC_w * S_w)) * (Cl_a_t / CL_a_w) - ((dCmf_dCL)))


# SM is the distance between CG and NP - lec 11 slide 42
# SM = (x_cg - x_25MAC) / (c_w) - (Cl_a_t * S_ht * L_ht) / (CL_a_w * S_w * c_w) + dCmf_dCL / CL_a_w

SM = (x_np - x_cg) / MAC_w
print("SM = " + str(SM))

#x_np = (-SM) * MAC_w + x_cg
print("Neutral point of the aircraft from the nose end (ft): ", str(np.round(x_np, 2)))


SM = 0.19925110752797284
Neutral point of the aircraft from the nose end (ft):  14.89
